In [17]:


orders_df=spark.read.format('parquet').load(
'abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/bronze/orders_data'
)
display(orders_df)

StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 19, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 1ffcca47-669e-43af-874f-89c64699717e)

In [18]:
from pyspark.sql.functions import *
orders_df=orders_df.withColumnRenamed('Order_ID','OrderId')\
.withColumnRenamed('cust_id','customerId')\
.withColumnRenamed('Product_Name','ProductName')\
.withColumnRenamed('Qty','Quantity')\
.withColumnRenamed('Order_Date','OrderDate')\
.withColumnRenamed('Order_Amount$','OrderAmount')\
.withColumnRenamed('Delivery_Status','DeliveryStatus')\
.withColumnRenamed('Payment_Mode','PaymentMode')\
.withColumnRenamed('Ship_Address','ShipAddress')\
.withColumnRenamed('Promo_code','PromoCode')\
.withColumnRenamed('Feedback_Score','FeedbackScore')\
.withColumn('ProductName',initcap(trim('ProductName')))
orders_df=orders_df.withColumn('Quantity',when(lower(col('Quantity')).isin('one'),lit(1))\
.otherwise(when(lower(col('Quantity'))=='three',lit(2))\
.otherwise(when(col('Quantity').rlike('^[0-9]+$'),col('Quantity'))\
.otherwise(lit(None))

)

))\
.withColumn('OrderAmount',regexp_replace('OrderAmount','[^0-9]',''))\
.withColumn('OrderAmount',col('OrderAmount').cast('Double'))\
.withColumn('DeliveryStatus',initcap('DeliveryStatus'))\
.withColumn('DeliveryStatus',when(col('DeliveryStatus').isin('Delivered','Delivrd'),'Delivered')\
.otherwise(col('DeliveryStatus')))\
.withColumn('PaymentMode',regexp_replace('PaymentMode','[^a-zA-Z]',''))\
.withColumn('PaymentMode',initcap('PaymentMode'))\
.withColumn('ShipAddress',regexp_replace('ShipAddress','[^0-9a-zA-z.,]',' '))\
.withColumn('ShipAddress',initcap('ShipAddress'))\
.withColumn('OrderDate',coalesce(
        to_date(col("OrderDate"), "dd/MM/yyyy"),
        to_date(col("OrderDate"), "dd-MM-yyyy"),
        to_date(col("OrderDate"), "dd.MM.yyyy"),
        to_date(col("OrderDate"), "MM.dd.yyyy"),
        to_date(col("OrderDate"), "MM-dd-yyyy"),
        to_date(col("OrderDate"), "MM/dd/yyyy"),
        to_date(col("OrderDate"), "yyyy/MM/dd"),
        to_date(col("OrderDate"), "yyyy.MM.dd"),
        to_date(col("OrderDate"), "yyyy/dd/MM")

)

)\
.withColumn('CustomerId',regexp_replace('CustomerId','[^a-zA-Z0-9]',''))\
.withColumn('PromoCode',upper('PromoCode'))\
.withColumn('FeedbackScore',when(col('FeedbackScore')=='bad',lit(2))\
.otherwise(col('FeedbackScore'))
)\
.withColumn('FeedbackScore',col('FeedbackScore').cast('double'))\
.withColumn('Email',when(col('Email').rlike('^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$'),col('Email'))\
.otherwise(lit(None))
)
orders_df.write.format('delta').mode('append').save(
        'Files/Silver/orders'
)

StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 20, Finished, Available, Finished, False)

In [19]:
inventory_df=spark.read.format('parquet').load('abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/bronze/inventory_data')
display(inventory_df)

StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 21, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, b0326166-367d-42b6-8969-74391cd7be55)

In [20]:
from pyspark.sql.functions import *
inventory_clean_df=inventory_df.withColumnRenamed('productName','ProductName')\
.withColumnRenamed('cost_price','CostPrice')\
.withColumnRenamed('last_stocked','LastStocked')
inventory_clean_df=inventory_clean_df.withColumn('stock',regexp_replace('stock','[^0-9]',''))\
.withColumn('stock',col('stock').cast('int'))\
.withColumn('CostPrice',regexp_replace(
    'CostPrice','[^0-9]',''
))\
.withColumn('CostPrice',col('CostPrice').cast('double'))\
.withColumn('warehouse',regexp_replace('warehouse','[^0-9a-zA-Z]',' '))\
.withColumn('available',when(lower(col('available')).isin('yes','y','true'),True)\
.otherwise(False)
)\
.withColumn('LastStocked',coalesce(to_date('LastStocked','yyyy-MM-dd'),\
to_date('LastStocked','dd-MM-yyyy'),
to_date('LastStocked','yyyy.MM.dd'),
to_date('LastStocked','yyyy/MM/dd'),
to_date('LastStocked','dd/MM/yyyy'),
to_date('LastStocked','yyyy-MM-dd'),
to_date('LastStocked','dd.MM.yyyy'),
to_date('LastStocked','MM/dd/yyyy')
))
inventory_clean_df.write.format('delta').mode('append').save(
        'Files/Silver/inventory'
)

StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 22, Finished, Available, Finished, False)

In [21]:
return_df=spark.read.format('parquet').load('abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/bronze/returns_data')
header = return_df.first()
return_df =return_df.toDF(*header).filter(col(header[0]) != header[0])
display(return_df)


StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 23, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, b93bf46a-ec8f-455d-90d8-3a5606ca7033)

In [22]:
return_clean_df=return_df.withColumn('Refund_status',regexp_replace('Refund_status','[^a-zA-Z]',''))\
.withColumn('Refund_status',lower('Refund_status'))\
.withColumn('Return_Amount',regexp_replace('Return_Amount','[^0-9]',''))\
.withColumn('Return_Amount',col('Return_Amount').cast('double'))\
.withColumn('Pickup_Address',regexp_replace('Pickup_Address','[^0-9A-Za-z]',' '))\
.withColumn('Product',regexp_replace('Product','[^0-9A-Za-z]',' '))\
.withColumn('Product',initcap('Product'))\
.withColumn('Customer_ID',regexp_replace('Customer_ID','[^0-9A-Za-z]',''
))\
.withColumn('Customer_ID',upper('Customer_ID'))\
.withColumn('Return_Date',coalesce(
    to_date('Return_Date','dd-MM-yyyy'),
    to_date('Return_Date','yyyy/MM/dd'),
    to_date('Return_Date','yyyy.MM.dd'),
    to_date('Return_Date','dd.MM.yyyy'),
    to_date('Return_Date','MM/dd/yyyy'),
     to_date('Return_Date','yyyy-MM-dd'),
     to_date('Return_Date','dd/MM/yyyy')
))
return_clean_df.write.format('delta').mode('append').save(
        'Files/Silver/returns'
)


StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 24, Finished, Available, Finished, True)

In [27]:
order_silver_df=spark.read.format('delta').load('abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/Silver/orders')
inventory_silver_df=spark.read.format('delta').load('abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/Silver/inventory')
returns_silver_df=spark.read.format('delta').load('abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/Silver/returns')
display(returns_silver_df)

StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 29, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, a2624e55-fda4-4578-bbfe-6a4c0f8e6be6)

In [35]:
combined_df=order_silver_df.join(returns_silver_df,order_silver_df.CustomerId==returns_silver_df.Customer_ID,how='left')\
.join(inventory_silver_df,order_silver_df.ProductName==inventory_silver_df.ProductName,how='left')\
.drop(returns_silver_df.Customer_ID,returns_silver_df.Product,inventory_silver_df.ProductName)
combined_df.write.format('delta').mode('append').save('Files/gold/order_360_dashboard')

StatementMeta(, 7cbbfcfd-49bd-4184-b2a2-e878d1017859, 37, Finished, Available, Finished, False)

In [1]:
gold_360_df=spark.read.format('delta').load('abfss://medallain_workspace@onelake.dfs.fabric.microsoft.com/medallain_lakehouse.Lakehouse/Files/gold/order_360_dashboard')
display(gold_360_df)

StatementMeta(, 66b327bd-bd6d-4e13-82b5-ada648f55775, 3, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, a9856df0-db19-4da9-b24f-42f8e468e135)

In [5]:
from pyspark.sql.functions import *
gold_aggregated_df=gold_360_df.groupBy('ProductName').agg(count('orderId').alias('order_count'),countDistinct('CustomerId').alias('customer_count'),
count('return_id').alias('return_count'),((count('return_id')/count('OrderId'))*100).alias('return_rate'),sum('OrderAmount').alias('total_revenue'),
avg('OrderAmount').alias('avg_order_amount'),sum('stock').alias('total_stock'),
avg('CostPrice').alias('avg_cost_price'),(sum('OrderAmount')-(avg('CostPrice')*sum('stock'))).alias('net_profit'))
gold_aggregated_df.write.format('delta').mode('append').save('Files/gold/aggregated_kpi_metrics')

StatementMeta(, 66b327bd-bd6d-4e13-82b5-ada648f55775, 7, Finished, Available, Finished, False)